# 1. Utilizando el dataset de object detection: Importando e dataset de conducion autonoma

In [1]:
import os
import pathlib
import random
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
!git clone https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 79704, done.
remote: Counting objects: 100% (410/410), done.
remote: Compressing objects: 100% (237/237), done.
remote: Total 79704 (delta 224), reused 327 (delta 169), pack-reused 79294
Receiving objects: 100% (79704/79704), 594.13 MiB | 2.24 MiB/s, done.
Resolving deltas: 100% (56672/56672), done.


In [3]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

bash: line 2: protoc: command not found


Processing /home/fredy/Documents/files/platzi/object_detection/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 5.9 MB/s eta 

ERROR: Exception:
Traceback (most recent call last):
  File "/home/fredy/.local/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 437, in _error_catcher
    yield
  File "/home/fredy/.local/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 560, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "/home/fredy/.local/lib/python3.10/site-packages/pip/_vendor/urllib3/response.py", line 526, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "/home/fredy/.local/lib/python3.10/site-packages/pip/_vendor/cachecontrol/filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "/home/fredy/anaconda3/envs/od/lib/python3.10/http/client.py", line 465, in read
    s = self.fp.read(amt)
  File "/home/fredy/anaconda3/envs/od/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "/home/fredy/anaconda3/envs/od/lib/python3.10/ssl.py", line 1274, in recv_into
    r

CalledProcessError: Command 'b'cd models/research/\nprotoc object_detection/protos/*.proto --python_out=.\ncp object_detection/packages/tf2/setup.py .\npython -m pip install .\n'' returned non-zero exit status 2.